Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from scipy.linalg import inv
import os

Importing Data from files

In [23]:

filepath_io_data_raw = r"C:\Users\devan\IO_data_EN2019.csv"
io_data_raw = pd.read_csv(filepath_io_data_raw)

# io_data_raw = io_data_raw.rename(columns={'X': 'sectors'})
# sector_labels = io_data_raw[['sectors']].to_dict(orient='list')



# Read the yearly cost data
filepath_results_for_io = r"C:\Users\devan\results_for_io.csv"
yearly_cost_data = pd.read_csv(filepath_results_for_io)

# Read the cost distribution data
filepath_cost_factors_renovation_IO = r"C:\Users\devan\cost_factors_renovation_IO.csv"
cost_distribution = pd.read_csv(filepath_cost_factors_renovation_IO)





Defining Parameter

In [20]:
count_sectors = 72  # Number of sectors in the IO table
location_final_consumption = 74  # Location of final consumption data
location_wages = 78  # Location of wages data

start_year = 2024  # Start year for the analysis
end_year = 2050  # End year for the analysis
duration = end_year - start_year + 1  # Duration of the analysis in years

Creating Sector labels as row indices

In [25]:
io_cost = io_data_raw.drop(columns=['sectors'])
io_cost.index = io_data_raw['sectors']


Creating transposed version of IO table

In [ ]:
io_demand = io_cost.T
io_cost_reduced = io_cost.iloc[:count_sectors, :count_sectors + 1]

Empty dataframes to collect impulse vectors

In [ ]:
io_impulse_empty = pd.DataFrame(np.zeros((duration, count_sectors + 1)), columns=list(io_demand.index) + ["year"])
io_impulse_empty["year"] = range(start_year, end_year + 1)

Distributing renovation costs across different economic sectors

In [ ]:
cost_distribution = cost_distribution.assign(
    share_installation=cost_distribution['share_roof'] * cost_distribution['roof_inst'] +
                       cost_distribution['share_wall'] * cost_distribution['wall_inst'] +
                       cost_distribution['share_ceiling'] * cost_distribution['ceiling_inst'] +
                       cost_distribution['share_window'] * cost_distribution['window_inst'],
    share_material=cost_distribution['share_roof'] * cost_distribution['roof_mat'] +
                   cost_distribution['share_wall'] * cost_distribution['wall_mat'] +
                   cost_distribution['share_ceiling'] * cost_distribution['ceiling_mat'],
    other_chemistry=cost_distribution['share_material'] * cost_distribution['share_chem'],
    other_ceramics=cost_distribution['share_material'] * cost_distribution['share_keramik'],
    other_glass=cost_distribution['share_material'] * cost_distribution['share_glass'],
    other_plastics=cost_distribution['window_mat'] * cost_distribution['share_window']
)

Creating Impulse vectors

In [ ]:
yearly_cost_redux = yearly_cost_data.rename(columns={'yearly_io_hp_cost': 'hp_cost'})[['year', 'io_cost_prio', 'hp_cost']]
yearly_cost_redux['other_costs'] = yearly_cost_redux['io_cost_prio'] - yearly_cost_redux['hp_cost']

yearly_cost_io = io_impulse_empty.iloc[:, :count_sectors + 1].copy()
yearly_cost_io = yearly_cost_io.assign(
    Specialised_construction_works=(
        yearly_cost_redux['other_costs'] * cost_distribution['share_installation'] +
        yearly_cost_redux['hp_cost'] * cost_distribution['hp_inst']
    ),
    Chemicals_and_chemical_products=(
        yearly_cost_redux['other_costs'] * cost_distribution['other_chemistry']
    ),
    Ceramic_products_processed_stone_and_clay=(
        yearly_cost_redux['other_costs'] * cost_distribution['other_ceramics']
    ),
    Glass_and_glassware=(
        yearly_cost_redux['other_costs'] * cost_distribution['other_glass']
    ),
    Rubber_and_plastics_products=(
        yearly_cost_redux['other_costs'] * cost_distribution['other_plastics']
    ),
    Electrical_equipment=(
        yearly_cost_redux['hp_cost'] * cost_distribution['hp_electricity']
    ),
    Machinery=(
        yearly_cost_redux['hp_cost'] * cost_distribution['hp_machines']
    ),
    Wages_and_salaries=cost_distribution['Wages.and.salaries']
)


Calculating the Leontief Inverse

In [ ]:
x = pd.concat([
    io_cost.loc[:, ['Total.uses.of.products']].iloc[:count_sectors],
    io_demand.rename(columns={'Wages.and.salaries': 'Total.uses.of.products'}).iloc[[location_wages - 1], ['Total.uses.of.products']]
], ignore_index=True)

x.index = list(io_cost.index[:count_sectors]) + ['Wages.and.salaries']


x_hat = np.diag(x['Total.uses.of.products'])


A = np.dot(io_cost_reduced.to_numpy(), inv(x_hat))

L_inverse = inv(np.eye(count_sectors + 1) - A)

Using Leontief Inverse to compute the economic output for each year

In [ ]:
new_output_matrix = np.dot(L_inverse, yearly_cost_io.iloc[:, 1:count_sectors + 2].T.to_numpy())


yearly_new_output = pd.DataFrame(new_output_matrix)


yearly_new_output.columns = range(start_year, end_year + 1)
yearly_new_output.index = yearly_cost_io.columns[1:count_sectors + 2]

Calculating Sector Impact

In [ ]:
sector_impact_employed = yearly_new_output.iloc[:count_sectors, :] * factors_and_aggregates['effect_employed'].values[:, np.newaxis]
sector_impact_employment = yearly_new_output.iloc[:count_sectors, :] * factors_and_aggregates['effect_employment'].values[:, np.newaxis]
sector_impact_imports = yearly_new_output.iloc[:count_sectors, :] * factors_and_aggregates['share_imports'].values[:, np.newaxis]
sector_impact_profit = yearly_new_output.iloc[:count_sectors, :] * factors_and_aggregates['share_profit'].values[:, np.newaxis]
sector_impact_wages = yearly_new_output.iloc[:count_sectors, :] * factors_and_aggregates['share_wages'].values[:, np.newaxis]
sector_impact_VA = yearly_new_output.iloc[:count_sectors, :] * factors_and_aggregates['share_VA'].values[:, np.newaxis]


In [ ]:
io_results = pd.DataFrame({
    'year': range(start_year, end_year + 1),
    'growth_employed': sector_impact_employed.sum(axis=0),
    'growth_employment': sector_impact_employment.sum(axis=0),
    'growth_imports': sector_impact_imports.sum(axis=0),
    'growth_profit': sector_impact_profit.sum(axis=0),
    'growth_wages': sector_impact_wages.sum(axis=0),
    'growth_VA': sector_impact_VA.sum(axis=0),
    'initial_stimulus': yearly_cost_io.iloc[:, 1:count_sectors + 2].sum(axis=1)
})


io_results['multiplier'] = io_results['growth_VA'] / io_results['initial_stimulus']
io_results = io_results[['year', 'multiplier'] + [col for col in io_results.columns if col not in ['year', 'multiplier']]]

io_results_mean = pd.DataFrame(io_results.iloc[:, 1:].mean(axis=0), columns=['mean']).T